In [8]:
import torch
from torch import nn
# 一、互相关运算
# 图像的卷积实际是互相关运算
def corr2d(X, K): # 计算二维互相关运算 X是输入 K是卷积核
    h, w = K.shape
    Y = torch.zeros(X.shape[0]-h+1, X.shape[1]-w+1) # 由于卷积核的宽度和高度大于1，所以输入大小变小
    for i in range(Y.shape[0]): # 卷积运算
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h, j:j+w] * K).sum() # torch中两个矩阵的*运算是按元素运算
    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
print(corr2d(X, K))

# 二、卷积层
"""
卷积层对输入和卷积核权重进行互相关运算，并在添加标量偏置之后产生输出。
所以，卷积层中的两个被训练的参数是卷积核权重和标量偏置。
就像之前随机初始化全连接层一样，在训练基于卷积层的模型时，也随机初始化卷积核权重
基于上面定义的corr2d函数实现二维卷积层。在__init__构造函数中,将weight和bias声明为两个模型参数。
前向传播函数调用corr2d函数并添加偏置

高度和宽度分别为h和w的卷积核可以被称为h*w卷积或h*w卷积核。
也将带有h*w卷积核的卷积层称为h*w卷积层。
"""
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(size=kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias # 加上偏置时应用了广播机制
    
# 三、图像中目标的边缘检测
X = torch.ones((6,8))
X[:, 2:6] = 0 # 构造一个6*8像素的黑白图像 中间四列为黑色0，其余为白色1
print(f"X:{X}")
kernel = torch.tensor([[1.0,-1.0]]) # 构造高度为1，宽度为2的卷积核,进行互相关运算时，如果水平相邻的两元素相同输出0，否则输出非0
Y = corr2d(X, kernel)
print(f"Y:{Y}")

# 四、卷积核
"""
查看“输入-输出”对 来学习 由X生成Y的卷积核。先构造一个卷积层,并将其卷积核初始化为随机张量。
接下来,在每次迭代中,比较Y与卷积层输出的平方误差,然后计算梯度来更新卷积核。
为了简单起见，在此使用内置的二维卷积层，并忽略偏置。
"""
conv2d = nn.Conv2d(in_channels=1, out_channels=1, 
                   kernel_size=(1,2), bias=False) # 构造二维卷积层 具有一个输出通道和形状为(1,2)的卷积核
# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))
learning_rate = 3e-2
for i in range(10):
    Y_hat = conv2d(X)
    loss = (Y_hat-Y)**2
    conv2d.zero_grad()
    loss.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= learning_rate*conv2d.weight.grad
    print(f"epoch:{i}, loss={loss.sum():.3f}")
print(conv2d.weight.data.reshape((1,2))) # 可以看到这个卷积核的权重已经和之前手动设置的边缘检测卷积核十分接近

tensor([[19., 25.],
        [37., 43.]])
X:tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])
Y:tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])
epoch:0, loss=19.584
epoch:1, loss=9.931
epoch:2, loss=5.290
epoch:3, loss=2.949
epoch:4, loss=1.708
epoch:5, loss=1.020
epoch:6, loss=0.623
epoch:7, loss=0.386
epoch:8, loss=0.242
epoch:9, loss=0.153
tensor([[ 0.9489, -1.0287]])
